# Mlpipeline
Feature engineering, train, evaluate model and generate submission 
- <a href='#1'>1. feature_engineering</a> 
- <a href='#2'>2. train</a> 
- <a href='#3'>3. predict</a>

In [1]:
! pip install xgboost -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com
%load_ext autoreload
%autoreload 2
import sys
import os
import gc
from time import time
from datetime import timedelta, datetime
import warnings
from collections import defaultdict
from tqdm import tqdm

import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import lightgbm as lgb
from sklearn import metrics
from dinglingling import wx_reminder

sys.path.append('../')
import conf
from mlpipeline import (
    feature_engineering_pandas,
    train,
    predict,
)
from utils import (
    check_columns,
    check_nan_value,
    load_model,
)

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/


In [2]:
# global settings
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns',1000)
pd.set_option('display.width',100)
sns.set(rc={'figure.figsize':(20,10)})
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
warnings.simplefilter('ignore', np.RankWarning)

In [3]:
# global variables
SCKEY = 'SCU92138T03d57ff9d4b08ced24c2cceb440cd3bd5e843242680de'  # used for reminding when feature engineering or model training completes

In [4]:
# functions
def __dummy():
    pass

@wx_reminder(SCKEY=SCKEY, remind_started=True)  
def feature_engineering_wrapper(params):
        """
        wrapper for feature engineering func 
        for reminding when it completes
        """
        train_fe_df, test_fe_df = feature_engineering_pandas(**params)
        
        return train_fe_df, test_fe_df    
    
@wx_reminder(SCKEY=SCKEY, remind_started=True)  
def train_wrapper(params):
        if params['is_eval']:
            _,_ = train(**params)
        else: 
            model, scaler = train(**params) 
            return model, scaler

In [5]:
! du -sh ../data/*

1.3G	../data/creative_id_w2v.bin
28M	../data/label_round_one_df.feather
1.2G	../data/raw_test_df.feather
1.2G	../data/raw_train_round_one_df.feather
13G	../data/tencent_2019_ad_competition_data
732M	../data/test
217M	../data/test.zip
771M	../data/test_fe_df.feather
715M	../data/train_fe_df.feather
637M	../data/train_preliminary
197M	../data/train_preliminary.zip


### <a id='1'> 1.feature_engineering</a>

In [6]:
# feature engineering
params = {
    'train_preprocessed_data_filename':'raw_train_round_one_df.feather', 
    'test_preprocessed_data_filename':'raw_test_df.feather', 
    'train_fe_save_filename': 'train_fe_df.feather',
    'test_fe_save_filename': 'test_fe_df.feather',
    'emb_method':'w2v',
    'max_df':0.9,  # param for tf_idf
    'min_df':3,  # param for tf_idf
    'emb_dim':128,
    'window':5,
    'min_count':1,
    'sample':6e-5,
    'negative':0,
    'hs':1,
    'alpha':0.03,
    'min_alpha':0.0007,
    'workers':20,
    'sg':1,
    'num_processes': 40,
    'is_train':True,
}

train_fe_df, test_fe_df = feature_engineering_wrapper(params)

2020-05-15 12:46:20,809 - mlpipeline.feature_engineering.feature_engineering - INFO - feature_engineering_pandas开始
2020-05-15 12:46:20,812 - mlpipeline.feature_engineering.feature_engineering - INFO - _load_preprocessed_data开始
2020-05-15 12:46:21,884 - mlpipeline.feature_engineering.feature_engineering - INFO - _load_preprocessed_data已完成，共用时0:00:01
2020-05-15 12:46:29,308 - mlpipeline.feature_engineering.feature_engineering - INFO - _generate_emb_for_sparse_feat开始
../mlpipeline/feature_engineering/feature_engineering.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sparse_feat_seq_df[sparse_feat] = sparse_feat_seq_df[sparse_feat].astype(str)
2020-05-15 12:52:32,259 - mlpipeline.feature_engineering.feature_engineering - INFO - sparse_feat: crea

In [6]:
train_fe_df.shape, test_fe_df.shape

NameError: name 'train_fe_df' is not defined

In [8]:
train_fe_df.head()

,user_id,w2v_1,w2v_2,w2v_3,w2v_4,w2v_5,w2v_6,w2v_7,w2v_8,w2v_9,w2v_10,w2v_11,w2v_12,w2v_13,w2v_14,w2v_15,w2v_16,w2v_17,w2v_18,w2v_19,w2v_20,w2v_21,w2v_22,w2v_23,w2v_24,w2v_25,w2v_26,w2v_27,w2v_28,w2v_29,w2v_30,w2v_31,w2v_32,w2v_33,w2v_34,w2v_35,w2v_36,w2v_37,w2v_38,w2v_39,w2v_40,w2v_41,w2v_42,w2v_43,w2v_44,w2v_45,w2v_46,w2v_47,w2v_48,w2v_49,w2v_50,w2v_51,w2v_52,w2v_53,w2v_54,w2v_55,w2v_56,w2v_57,w2v_58,w2v_59,w2v_60,w2v_61,w2v_62,w2v_63,w2v_64,w2v_65,w2v_66,w2v_67,w2v_68,w2v_69,w2v_70,w2v_71,w2v_72,w2v_73,w2v_74,w2v_75,w2v_76,w2v_77,w2v_78,w2v_79,w2v_80,w2v_81,w2v_82,w2v_83,w2v_84,w2v_85,w2v_86,w2v_87,w2v_88,w2v_89,w2v_90,w2v_91,w2v_92,w2v_93,w2v_94,w2v_95,w2v_96,w2v_97,w2v_98,w2v_99,w2v_100,age,gender,y
0,1,0.317886,-0.526415,-0.055048,-0.528955,-0.096361,0.162796,0.048159,0.119717,-0.040841,-0.436568,-0.032624,0.112490,-0.325787,0.138070,-0.241391,-0.253797,0.129697,0.532830,-0.220061,-0.047494,0.281527,-0.292721,-0.180316,-0.217422,-0.299926,-0.409515,0.536922,-0.150833,0.608378,0.519931,0.327593,-0.047129,-0.153875,-0.308788,-0.009804,-0.284649,0.363233,-0.126311,-0.099959,-0.399930,-0.008959,-0.557281,0.208298,0.002441,0.229310,-0.095422,0.478953,0.017111,-0.488578,0.167485,0.055130,-0.015642,0.154260,0.166007,-0.038812,-0.298765,-0.494071,-0.282491,0.016670,-0.373491,-0.480105,0.116221,-0.369969,0.233782,-0.427903,-0.053939,0.015587,0.521378,-0.069436,-0.167746,-0.121342,0.047510,-0.453079,-0.044746,-0.312390,0.092713,-0.270386,0.118622,0.128435,0.047801,0.021427,-0.179088,-0.134783,0.039494,-0.272888,-0.009532,0.177008,0.685854,-0.450034,0.525002,0.347635,0.345707,0.180106,-0.140729,-0.212545,0.388303,-0.458613,-0.068500,-0.613453,0.725146,4,1,3
1,2,0.325203,-0.367411,-0.279791,-0.093693,-0.122351,0.297370,-0.206585,0.076459,0.294193,-0.571907,-0.055274,0.130691,-0.330579,0.090058,-0.629760,-0.099451,0.191704,0.427102,-0.219477,-0.062363,-0.117611,-0.434651,0.044292,-0.188073,0.070248,-0.450696,0.693025,0.267765,0.488802,0.422140,0.118047,-0.051940,-0.085290,-0.212455,0.080108,-0.335424,0.391596,0.204405,-0.231755,-0.379702,0.040530,-0.422194,0.579996,-0.274050,-0.108324,0.196841,0.737161,0.418932,-0.240259,0.285742,-0.203938,0.192743,0.287305,-0.290213,-0.273336,0.011194,-0.399577,-0.422319,-0.018834,-0.419183,-0.187218,0.080601,-0.404032,-0.018926,-0.800364,0.318652,-0.158947,0.312360,-0.095327,-0.231974,-0.330422,0.121357,-0.249703,0.121524,-0.148256,0.157988,-0.415097,0.073084,0.003465,-0.329378,0.122206,-0.130705,-0.119785,0.072925,-0.366768,-0.099851,0.126987,0.645205,-0.032412,0.242342,0.211180,-0.006131,-0.254851,0.070728,-0.232397,-0.214553,-0.202904,0.094728,-0.275755,0.398690,10,1,9
2,3,0.198565,-0.586632,-0.262671,0.027136,0.100215,0.322511,-0.073465,0.222631,0.041526,-0.586876,-0.160328,0.126489,-0.086501,-0.182446,-0.321912,0.072478,0.058277,0.273296,0.040683,-0.374089,-0.047131,0.089502,-0.090019,-0.217124,0.050214,-0.337977,0.150414,0.182220,0.169077,0.158828,0.159270,0.144751,-0.066186,-0.040247,-0.032578,-0.190561,0.008836,0.151353,0.133934,-0.368623,0.134417,-0.560351,0.443211,-0.094252,-0.098049,-0.137316,0.347831,0.367783,-0.088025,0.299211,0.222399,0.057067,-0.206091,0.319835,-0.401369,-0.390524,-0.407110,-0.227170,-0.037186,-0.336939,-0.046157,-0.171637,-0.315089,0.435404,-0.487106,-0.021420,0.050940,0.625776,-0.113916,-0.152975,-0.259297,0.502868,-0.481878,0.038209,-0.421211,-0.215066,-0.276196,0.333828,0.009564,0.081381,-0.134869,-0.039792,-0.095843,0.164616,-0.148636,0.063577,0.637608,0.581705,-0.248123,0.566989,0.145143,-0.153065,0.301695,-0.154103,-0.555241,-0.109893,-0.180794,-0.198200,-0.214605,0.468438,7,2,16
3,4,0.151257,-0.374335,-0.428679,-0.272160,0.215631,0.185658,-0.006619,0.316194,0.247443,-0.474306,-0.171284,-0.099072,-0.241625,-0.016837,-0.539686,0.092095,0.188419,0.393323,0.016091,-0.133066,-0.085917,-0.090432,-0.349310,-0.248880,-0.120003,-0.250547,0.428762,0.223570,0.324844,0.603414,0.053672,0.051973,-0.119606,-0.243262,-0.225287,-0.183186,-0.009053,0.117919,0.068445,-0.254893,

In [ ]:
# eval 
# lgb_model_params = {
#                'objective': 'multiclass',  # multiclass, binary 
#                'boosting': 'gbdt',
#                'learning_rate': 0.15,
#                'metric': ['multi_logloss'],  # 'binary_logloss', 'multi_logloss'
#                'num_threads': 15,
#                'random_state': 2019,
#                'num_boost_round': 1000,
#                'device': 'cpu',
#                'num_class':20,  # 2, 20 ,10
#                'num_leaves':32,  # [16,32,64,128]
#                'subsample': 0.9,  # [0.7,0.8,0.9,1]
#                'colsample_bytree': 0.9, # [0.2,0.3,0.4,0.5,0.6]
#                'min_data_in_leaf': 40, # [20,40,60,80,100]
#                'lambda_l1': 1.0,  # (0.2,3)
#                'lambda_l2': 1.0,  # (0.2,3)
# }

xgb_model_params = {
               'objective': 'multi:softmax',  # multiclass, binary 
               'booster': 'gbtree',
               'eta': 0.15,
               'eval_metric': ['mlogloss'],  # 'binary_logloss', 'multi_logloss'
               'nthread': 15,
               'random_state': 2019,
               'tree_method':'auto',
               'n_estimators': 1000,
               'device': 'cpu',
               'num_class':20,  # 2, 20 ,10
               'max_leaves':32,  # [16,32,64,128]
               'subsample': 0.9,  # [0.7,0.8,0.9,1]
               'colsample_bytree': 0.9, # [0.2,0.3,0.4,0.5,0.6]
               'min_data_in_leaf': 40, # [20,40,60,80,100]
               'lambda': 1.0,  # (0.2,3)
               'alpha': 1.0,  # (0.2,3)
}

params = {
    'fe_filename':'train_fe_df.feather', 
    'is_eval':True, 
    'model_type': 'ensemble',
    'model_name': 'xgb',
    'model_params': xgb_model_params,
    'use_log': False,
    'use_std': True,
    'use_cv': True,  
    'n_splits':2,  
}
train_wrapper(params)

2020-05-15 16:36:20,286 - mlpipeline.train - INFO - using_fe_df: train_fe_df.feather, use_label: y, is_eval: True, model_type: ensemble, model_name: xgb, use_log: False, use_std: True, use_cv: True, n_splits: 2
2020-05-15 16:36:20,451 - mlpipeline.train - INFO - _train_pipeline_ensemble_and_linear开始
2020-05-15 16:36:20,456 - mlpipeline.train - INFO - 连续性特征数量: 100
2020-05-15 16:36:20,457 - mlpipeline.train - INFO - 离散性特征数量: 0
2020-05-15 16:36:21,895 - mlpipeline.train - INFO - eval参数: {'objective': 'multi:softmax', 'booster': 'gbtree', 'eta': 0.15, 'eval_metric': ['mlogloss'], 'nthread': 15, 'random_state': 2019, 'tree_method': 'auto', 'n_estimators': 1000, 'device': 'cpu', 'num_class': 20, 'max_leaves': 32, 'subsample': 0.9, 'colsample_bytree': 0.9, 'min_data_in_leaf': 40, 'lambda': 1.0, 'alpha': 1.0}
2020-05-15 16:36:21,896 - mlpipeline.train - INFO - _get_cv_folds开始
2020-05-15 16:36:21,898 - mlpipeline.train - INFO - _get_cv_folds已完成，共用时0:00:00
2020-05-15 16:36:23,070 - utils.utils -

[0]	validation_0-mlogloss:2.83766	validation_1-mlogloss:2.84314
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn't improved in 10 rounds.
[1]	validation_0-mlogloss:2.72185	validation_1-mlogloss:2.73148
[2]	validation_0-mlogloss:2.62847	validation_1-mlogloss:2.64185
[3]	validation_0-mlogloss:2.55161	validation_1-mlogloss:2.56833
[4]	validation_0-mlogloss:2.48569	validation_1-mlogloss:2.50554
[5]	validation_0-mlogloss:2.42916	validation_1-mlogloss:2.45206
[6]	validation_0-mlogloss:2.37942	validation_1-mlogloss:2.40511
[7]	validation_0-mlogloss:2.33567	validation_1-mlogloss:2.36424
[8]	validation_0-mlogloss:2.29666	validation_1-mlogloss:2.32785
[9]	validation_0-mlogloss:2.26168	validation_1-mlogloss:2.29547
[10]	validation_0-mlogloss:2.22982	validation_1-mlogloss:2.26611
[11]	validation_0-mlogloss:2.20113	validation_1-mlogloss:2.23992
[12]	validation_0-mlogloss:2.17512	validation_1-mlogloss:2.2163

[124]	validation_0-mlogloss:1.57153	validation_1-mlogloss:1.82722
[125]	validation_0-mlogloss:1.56925	validation_1-mlogloss:1.82665
[126]	validation_0-mlogloss:1.56703	validation_1-mlogloss:1.82604
[127]	validation_0-mlogloss:1.56489	validation_1-mlogloss:1.82545
[128]	validation_0-mlogloss:1.56277	validation_1-mlogloss:1.82491
[129]	validation_0-mlogloss:1.56063	validation_1-mlogloss:1.82436
[130]	validation_0-mlogloss:1.55856	validation_1-mlogloss:1.82385
[131]	validation_0-mlogloss:1.55650	validation_1-mlogloss:1.82343
[132]	validation_0-mlogloss:1.55444	validation_1-mlogloss:1.82292
[133]	validation_0-mlogloss:1.55245	validation_1-mlogloss:1.82238
[134]	validation_0-mlogloss:1.55032	validation_1-mlogloss:1.82181
[135]	validation_0-mlogloss:1.54823	validation_1-mlogloss:1.82130
[136]	validation_0-mlogloss:1.54629	validation_1-mlogloss:1.82083
[137]	validation_0-mlogloss:1.54435	validation_1-mlogloss:1.82038
[138]	validation_0-mlogloss:1.54243	validation_1-mlogloss:1.81989
[139]	vali

[249]	validation_0-mlogloss:1.36316	validation_1-mlogloss:1.79053
[250]	validation_0-mlogloss:1.36187	validation_1-mlogloss:1.79039
[251]	validation_0-mlogloss:1.36045	validation_1-mlogloss:1.79024
[252]	validation_0-mlogloss:1.35910	validation_1-mlogloss:1.79011
[253]	validation_0-mlogloss:1.35776	validation_1-mlogloss:1.78998
[254]	validation_0-mlogloss:1.35641	validation_1-mlogloss:1.78985
[255]	validation_0-mlogloss:1.35508	validation_1-mlogloss:1.78972
[256]	validation_0-mlogloss:1.35376	validation_1-mlogloss:1.78962
[257]	validation_0-mlogloss:1.35242	validation_1-mlogloss:1.78948
[258]	validation_0-mlogloss:1.35118	validation_1-mlogloss:1.78933
[259]	validation_0-mlogloss:1.34981	validation_1-mlogloss:1.78921
[260]	validation_0-mlogloss:1.34843	validation_1-mlogloss:1.78908
[261]	validation_0-mlogloss:1.34717	validation_1-mlogloss:1.78893
[262]	validation_0-mlogloss:1.34577	validation_1-mlogloss:1.78881
[263]	validation_0-mlogloss:1.34450	validation_1-mlogloss:1.78872
[264]	vali

In [16]:
# train 
model_params = {
               'objective': 'multiclass',  # multiclass, binary 
               'boosting': 'gbdt',
               'learning_rate': 0.15,
               'metric': ['multi_logloss'],  # 'binary_logloss', 'multi_logloss'
               'num_threads': 20,
               'random_state': 2019,
               'num_boost_round': 620,
               'device': 'cpu',
               'num_class':20,  # 2, 20 ,10
               'num_leaves':32,  # [16,32,64,128]
               'subsample': 0.9,  # [0.7,0.8,0.9,1]
               'colsample_bytree': 0.9, # [0.2,0.3,0.4,0.5,0.6]
               'min_data_in_leaf': 40, # [20,40,60,80,100]
               'lambda_l1': 1.0,  # (0.2,3)
               'lambda_l2': 1.0,  # (0.2,3)
}


params = {
    'fe_filename':'train_fe_df.feather', 
    'is_eval':False, 
    'model_type': 'ensemble',
    'model_name': 'xgb',
    'model_params': model_params,
    'use_log': False,
    'use_std': True,
}
model, scaler = train_wrapper(params)

2020-05-15 15:18:14,504 - mlpipeline.train - INFO - using_fe_df: train_fe_df.feather, use_label: y, is_eval: False, model_type: ensemble, model_name: lgb, use_log: False, use_std: True, use_cv: False, n_splits: 2
2020-05-15 15:18:14,680 - mlpipeline.train - INFO - _train_pipeline_ensemble_and_linear开始
2020-05-15 15:18:14,684 - mlpipeline.train - INFO - 连续性特征数量: 100
2020-05-15 15:18:14,684 - mlpipeline.train - INFO - 离散性特征数量: 0
2020-05-15 15:18:15,241 - mlpipeline.train - INFO - train参数:{'objective': 'multiclass', 'boosting': 'gbdt', 'learning_rate': 0.15, 'metric': ['multi_logloss'], 'num_threads': 20, 'random_state': 2019, 'num_boost_round': 620, 'device': 'cpu', 'num_class': 20, 'num_leaves': 32, 'subsample': 0.9, 'colsample_bytree': 0.9, 'min_data_in_leaf': 40, 'lambda_l1': 1.0, 'lambda_l2': 1.0}
2020-05-15 15:18:15,242 - utils.utils - INFO - standard_scale开始
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a

[1]	training's multi_logloss: 2.57176
Training until validation scores don't improve for 10 rounds.
[2]	training's multi_logloss: 2.5083
[3]	training's multi_logloss: 2.4552
[4]	training's multi_logloss: 2.40947
[5]	training's multi_logloss: 2.36945
[6]	training's multi_logloss: 2.33421
[7]	training's multi_logloss: 2.3024
[8]	training's multi_logloss: 2.27397
[9]	training's multi_logloss: 2.248
[10]	training's multi_logloss: 2.22435
[11]	training's multi_logloss: 2.20279
[12]	training's multi_logloss: 2.18303
[13]	training's multi_logloss: 2.16459
[14]	training's multi_logloss: 2.14777
[15]	training's multi_logloss: 2.13202
[16]	training's multi_logloss: 2.11719
[17]	training's multi_logloss: 2.10366
[18]	training's multi_logloss: 2.09074
[19]	training's multi_logloss: 2.07859
[20]	training's multi_logloss: 2.06732
[21]	training's multi_logloss: 2.05667
[22]	training's multi_logloss: 2.04649
[23]	training's multi_logloss: 2.03683
[24]	training's multi_logloss: 2.02773
[25]	training's 

[208]	training's multi_logloss: 1.65259
[209]	training's multi_logloss: 1.65173
[210]	training's multi_logloss: 1.65085
[211]	training's multi_logloss: 1.64999
[212]	training's multi_logloss: 1.64913
[213]	training's multi_logloss: 1.64828
[214]	training's multi_logloss: 1.64743
[215]	training's multi_logloss: 1.64658
[216]	training's multi_logloss: 1.64575
[217]	training's multi_logloss: 1.64491
[218]	training's multi_logloss: 1.6441
[219]	training's multi_logloss: 1.64326
[220]	training's multi_logloss: 1.64243
[221]	training's multi_logloss: 1.64161
[222]	training's multi_logloss: 1.64076
[223]	training's multi_logloss: 1.63994
[224]	training's multi_logloss: 1.63913
[225]	training's multi_logloss: 1.63831
[226]	training's multi_logloss: 1.63749
[227]	training's multi_logloss: 1.63667
[228]	training's multi_logloss: 1.63586
[229]	training's multi_logloss: 1.63504
[230]	training's multi_logloss: 1.63425
[231]	training's multi_logloss: 1.63346
[232]	training's multi_logloss: 1.63264
[

[414]	training's multi_logloss: 1.51358
[415]	training's multi_logloss: 1.51304
[416]	training's multi_logloss: 1.51248
[417]	training's multi_logloss: 1.51193
[418]	training's multi_logloss: 1.51139
[419]	training's multi_logloss: 1.51082
[420]	training's multi_logloss: 1.51027
[421]	training's multi_logloss: 1.5097
[422]	training's multi_logloss: 1.50914
[423]	training's multi_logloss: 1.50861
[424]	training's multi_logloss: 1.50811
[425]	training's multi_logloss: 1.50756
[426]	training's multi_logloss: 1.50697
[427]	training's multi_logloss: 1.50639
[428]	training's multi_logloss: 1.50584
[429]	training's multi_logloss: 1.50529
[430]	training's multi_logloss: 1.50474
[431]	training's multi_logloss: 1.50422
[432]	training's multi_logloss: 1.50368
[433]	training's multi_logloss: 1.50316
[434]	training's multi_logloss: 1.50261
[435]	training's multi_logloss: 1.50204
[436]	training's multi_logloss: 1.50147
[437]	training's multi_logloss: 1.50095
[438]	training's multi_logloss: 1.50043
[

[620]	training's multi_logloss: 1.41037
Did not meet early stopping. Best iteration is:
[620]	training's multi_logloss: 1.41037


2020-05-15 15:31:20,213 - mlpipeline.train - INFO - _train_pipeline_ensemble_and_linear已完成，共用时0:13:06
2020-05-15 15:31:20,214 - mlpipeline.train - INFO - lgb模型训练完成!模型保存至:../trained_models/lgb.model.2020-05-15T15:18:14.680759


In [18]:
# predict 
params = {
          'test_fe_filename':'test_fe_df.feather',
          'use_log':False,
          'use_std': True,
          'model_type': 'ensemble',
          'model_name':'lgb',
          'scaler': scaler  # scaler
            }

submission_df = predict(**params)

2020-05-15 15:35:07,734 - mlpipeline.predict - INFO - predict开始
2020-05-15 15:35:07,735 - mlpipeline.predict - INFO - test_fe_filename: test_fe_df.feather, use_log: False, use_std: True, model_type: ensemble, model_name: lgb
2020-05-15 15:35:07,847 - mlpipeline.predict - INFO - inference_pipeline_ensemble_and_linear开始
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pa

In [19]:
submission_df.head()

,user_id,predicted_gender,predicted_age
0,3000001,1,3
1,3000002,2,7
2,3000003,2,2
3,3000004,1,2
4,3000005,1,3
